In [14]:
import pandas as pd
import os
import json


from IPython.display import display

## get files:

In [15]:
questionare_path = "../res/user_data/questionare.xlsx"
participant_order_path = "../res/participant_order.xlsx"
user_data_path = "../res/user_data/"

chat_bot_files  = [f for f in os.listdir(user_data_path) if f.endswith('.json')]

df_questionare = pd.read_excel(open(questionare_path, 'rb'), index_col=0)
df_participant_order = pd.read_excel(open(participant_order_path, 'rb'), index_col=0)



## questionare data preparation:

In [16]:
question_labels = ["The chatbot’s personality was realistic and engaging",
                   "The chatbot seemed too robotic",
                   "The chatbot was welcoming during initial setup",
                   "The chatbot seemed very unfriendly",
                   "The chatbot explained its scope and purpose well",
                   "The chatbot gave no indication as to its purpose",
                   "The chatbot was easy to navigate","It would be easy to get confused when using the chatbot",
                   "The chatbot understood me well","The chatbot failed to recognise a lot of my inputs",
                   "Chatbot responses were useful, appropriate and informative","Chatbot responses were irrelevant",
                   "The chatbot coped well with any errors or mistakes",
                   "The chatbot seemed unable to handle any errors",
                   "The chatbot was very easy to use",
                   "The chatbot was very complex"
                   ]
question_labels_2 = [x + "2" for x in question_labels]
rename_dict = {v: k for k, v in zip(question_labels, question_labels_2)}



In [17]:
df = df_questionare.merge(df_participant_order, left_on="Participant code", right_on="Participant number")
df.drop(columns=["Please complete the first three trials", "Please complete the last three trials", 'Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6'], inplace=True)
df.drop(columns=["​Experiment Description\n\nThe aim of the project is to determine how the communication style of a chatbot impacts the perceived quality of it. The communication styles vary on formality of language..."], inplace=True)
#print(df.columns)

In [18]:
# create df for genz
df_questionare_genz_1 = df[df["Chatbot group start"].str.match("Informal")]
df_questionare_genz_1 = df_questionare_genz_1.drop(columns=question_labels_2)
df_questionare_genz_2 = df[df["Chatbot group start"].str.match("Formal")]
df_questionare_genz_2 = df_questionare_genz_2.drop(columns=question_labels)
df_questionare_genz_2  = df_questionare_genz_2.rename(columns=rename_dict)


df_questionare_genz = pd.concat([df_questionare_genz_1, df_questionare_genz_2 ])
df_questionare_genz["language"] = "genz"
print(len(df_questionare_genz)) 


1


In [19]:
# same as above get df for formal
df_questionare_formal_1 = df[df["Chatbot group start"].str.match("Formal")]
df_questionare_formal_1 = df_questionare_formal_1.drop(columns=question_labels_2)
df_questionare_formal_2 = df[df["Chatbot group start"].str.match("Informal")]
df_questionare_formal_2 = df_questionare_formal_2.drop(columns=question_labels)
df_questionare_formal_2 = df_questionare_formal_2.rename(columns=rename_dict)

df_questionare_formal = pd.concat([df_questionare_formal_1, df_questionare_formal_2 ])
df_questionare_formal = df_questionare_formal.drop(columns=["Chatbot group start"])
df_questionare_formal["language"] = "formal"
#print((df_questionare_formal.columns)) 

In [20]:
df_questionare = pd.concat([df_questionare_genz, df_questionare_formal])
df_questionare = df_questionare.set_index(["Participant code", "language"])
df_questionare = df_questionare.drop(columns=["Chatbot group start"])


In [21]:
# 'Chatbot group start' gives the order of the user did the language options

In [22]:
def calc_CUQ_score(df):
    # step 1:
    agree_scores = {
        "Strongly agree": 5,
        "Agree": 4,
        "Neutral": 3,
        "Disagree": 2,
        "Strongly disagree": 1
    }
    # step 2 and 3:
    # odd and even are swapped, cause we count from 0
    # step 2
    positiv_score = 0
    for i in range(0,len(question_labels), 2):
        positiv_score += agree_scores[df[question_labels[i]]]
    # step 3:
    negative_score = 0
    for i in range(1,len(question_labels), 2):
        negative_score += agree_scores[df[question_labels[i]]]
    
    #step 4:
    positiv_score = positiv_score - 8
    # step 5:
    negative_score = 40 - negative_score
    # step 6:
    cuq_score = positiv_score + negative_score 
    #You should now have a score out of 64 
    if cuq_score > 64:
        raise ValueError("CUQ score is greater than 64")
    # step 7:
    cuq_score = cuq_score / 64 
    cuq_score = cuq_score * 100
    return cuq_score


for index, row in df_questionare.iterrows():
    print( calc_CUQ_score(row))
    df_questionare.at[index, "CUQ"] = calc_CUQ_score(row)




37.5
51.5625


In [23]:
display(df_questionare)

Start time     Completion time      Email  \
Participant code language                                                      
6                genz     2023-10-24 14:32:05 2023-10-24 14:47:12  anonymous   
                 formal   2023-10-24 14:32:05 2023-10-24 14:47:12  anonymous   

                           Name  Last modified time  Age  \
Participant code language                                  
6                genz       NaN                 NaN   23   
                 formal     NaN                 NaN   23   

                          The chatbot’s personality was realistic and engaging  \
Participant code language                                                        
6                genz                                                  Agree     
                 formal                                             Disagree     

                          The chatbot seemed too robotic  \
Participant code language                                  
6                genz                     Strongly agree   
                 formal                   Strongly agree   

                          The chatbot was welcoming during initial setup  \
Participant code language                                                  
6                genz                                              Agree   
                 formal                                          Neutral   

                          The chatbot seemed very unfriendly  ...  \
Participant code language                                     ...   
6                genz                      Strongly disagree  ...   
                 formal                    Strongly disagree  ...   

                          It would be easy to get confused when using the chatbot  \
Participant code language                                                           
6                genz                                         Strongly agree        
                 formal                                                Agree        

                          The chatbot understood me well  \
Participant code language                                  
6                genz                           Disagree   
                 formal                         Disagree   

                          The chatbot failed to recognise a lot of my inputs  \
Participant code language                                                      
6                genz                                         Strongly agree   
                 formal                                                Agree   

                          Chatbot responses were useful, appropriate and informative  \
Participant code language                                                              
6                genz                                                Neutral           
                 formal                                                Agree           

                          Chatbot responses were irrelevant  \
Participant code language                                     
6                genz                                 Agree   
                 formal                            Disagree   

                          The chatbot coped well with any errors or mistakes  \
Participant code language                                                      
6                genz                                      Strongly disagree   
                 formal                                             Disagree   

                          The chatbot seemed unable to handle any errors  \
Participant code language                                                  
6                genz                                     Strongly agree   
                 formal                                          Neutral   

                          The chatbot was very easy to use  \
Participant code language                                    
6                genz                             Disagree   


In [24]:
# user id

# total time (average of tasks with same language)
# number of suggestions (average of tasks with same language)
# number of turns (average of tasks with same language)
# number_of_suggestions
# number_of_restarts
df_chat = pd.DataFrame(columns=["user_id",  "language",  "number_of_turns", "time", "number_of_suggestions", "number_of_restarts"])
for chat_data in chat_bot_files:
    chat_data = json.load(open(user_data_path + chat_data))
    row = df_chat.loc[(df_chat["user_id"] == chat_data["user_id"] ) & (df_chat["language"] == chat_data["configuration"]["language"]  ) ]
    total_time = sum([x["response_seconds"] for x in chat_data["turns"]])
    if row.empty:

        row = pd.DataFrame([[ chat_data["user_id"],  chat_data["configuration"]["language"],  len(chat_data["turns"]) , total_time, chat_data["number_suggestions"], chat_data["number_restart"] ] ], 
                           columns=["user_id",  "language", "number_of_turns", "time", "number_of_suggestions", "number_of_restarts"])

        df_chat = pd.concat([df_chat, row])
    else:
        row.loc[0,"number_of_turns"] = row.loc[0,"number_of_turns"] + len(chat_data["turns"])
        row.loc[0,"time"] = row.loc[0,"time"] + total_time
        row.loc[0,"number_of_suggestions"] = row.loc[0,"number_of_suggestions"] + chat_data["number_suggestions"]
        row.loc[0, "number_of_restarts"] = row.loc[0, "number_of_restarts"] + chat_data["number_restart"]
        df_chat.loc[(df_chat["user_id"] == chat_data["user_id"] ) & (df_chat["language"] == chat_data["configuration"]["language"]  ) ] = row

df_chat["number_of_turns"] = df_chat["number_of_turns"] / 3
df_chat["time"] = df_chat["time"] / 3
df_chat["number_of_suggestions"] = df_chat["number_of_suggestions"] / 3
df_chat["number_of_restarts"] = df_chat["number_of_restarts"] / 3



df_chat["language"] = df_chat['language'].str.lower()
#df_chat.rename(columns={"user_id": "Participant code"}, inplace=True)
df_chat.set_index(["user_id", "language"], inplace=True)

display(df_chat)

number_of_turns       time number_of_suggestions  \
user_id language                                                    
6       genz           12.333333  83.626476                   1.0   
        formal          9.666667  80.554383                   1.0   
1       formal          5.666667  24.829856              0.333333   

                 number_of_restarts  
user_id language                     
6       genz               1.333333  
        formal             0.333333  
1       formal             0.333333

In [25]:
df_chat.reset_index(inplace=True)
df_questionare.reset_index(inplace=True)

In [26]:

df_questionare["index"] = df_questionare["Participant code"].astype(str) + df_questionare["language"]
df_chat["index"] = df_chat["user_id"] + df_chat["language"]
df = df_questionare.merge(df_chat, left_on=["index"], right_on=["index"])
df = df.drop(columns=["index", "language_y"] )
df = df.rename(columns={"language_x": "language"})
df.set_index(["Participant code", "language"], inplace=True)
display(df)


Start time     Completion time      Email  \
Participant code language                                                      
6                genz     2023-10-24 14:32:05 2023-10-24 14:47:12  anonymous   
                 formal   2023-10-24 14:32:05 2023-10-24 14:47:12  anonymous   

                           Name  Last modified time  Age  \
Participant code language                                  
6                genz       NaN                 NaN   23   
                 formal     NaN                 NaN   23   

                          The chatbot’s personality was realistic and engaging  \
Participant code language                                                        
6                genz                                                  Agree     
                 formal                                             Disagree     

                          The chatbot seemed too robotic  \
Participant code language                                  
6                genz                     Strongly agree   
                 formal                   Strongly agree   

                          The chatbot was welcoming during initial setup  \
Participant code language                                                  
6                genz                                              Agree   
                 formal                                          Neutral   

                          The chatbot seemed very unfriendly  ...  \
Participant code language                                     ...   
6                genz                      Strongly disagree  ...   
                 formal                    Strongly disagree  ...   

                          The chatbot coped well with any errors or mistakes  \
Participant code language                                                      
6                genz                                      Strongly disagree   
                 formal                                             Disagree   

                          The chatbot seemed unable to handle any errors  \
Participant code language                                                  
6                genz                                     Strongly agree   
                 formal                                          Neutral   

                          The chatbot was very easy to use  \
Participant code language                                    
6                genz                             Disagree   
                 formal                              Agree   

                          The chatbot was very complex      CUQ user_id  \
Participant code language                                                 
6                genz                         Disagree  37.5000       6   
                 formal                       Disagree  51.5625       6   

                          number_of_turns       time number_of_suggestions  \
Participant code language                                                    
6                genz           12.333333  83.626476                   1.0   
                 formal          9.666667  80.554383                   1.0   

                          number_of_restarts  
Participant code language                     
6                genz               1.333333  
                 formal             0.333333  

[2 rows x 28 columns]